<a href="https://colab.research.google.com/github/jairoRibeiro03/CovidBr-Data-Visualization/blob/master/visualizandoDadosCovid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# Habilita modo offline
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
# Definindo o renderizador
import plotly.io as pio
pio.renderers
pio.renderers.default = 'colab'
import plotly.express as px
import numpy as np

In [ ]:
file_covid = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-total.csv'
file_cidades = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities-time.csv'

In [ ]:
casos_totais = pd.read_csv(file_covid, sep=',', parse_dates=['date'])
cidades = pd.read_csv(file_cidades, sep=',', parse_dates=['date'])

In [ ]:
casos_totais.head()

In [ ]:
cidades.tail()

In [ ]:
cidades.isnull().sum()

In [ ]:
casos_totais.head()

In [ ]:
cidades.info()

In [ ]:
cidades.tail()

In [ ]:
cidades.head()

# Criando uma coluna chamada "Actives" para representar os casos ativos com a diferença entre os casos totais, mortes e recuperados.

In [ ]:
casos_totais['actives'] = casos_totais['totalCasesMS']-casos_totais['recovered']-casos_totais['deathsMS']

In [ ]:
casos_totais.isnull().sum()

Substituindo as colunas com *misssing values* (NaN) por 0

In [ ]:
casos_totais[['recovered', 'suspects', 'tests', 'tests_per_100k_inhabitants', 'actives']] = casos_totais[['recovered', 'suspects', 'tests', 'tests_per_100k_inhabitants', 'actives']].fillna(0)

In [ ]:
casos_totais.isnull().sum()

In [ ]:
cidades.head()

In [ ]:
casos_totais[['recovered', 'actives']] = casos_totais[['recovered', 'actives']].astype(int)

In [ ]:
casos_totais.shape

(28, 19)

In [ ]:
cidades.shape

(110707, 13)

##Analisando dados temporais

In [ ]:
casos_totais.head()

In [ ]:
casos_totais.date.describe()

In [ ]:
cidades.tail()

In [ ]:
# Cores 
total = '#1B08C7'
recuperados = '#21bf73'
mortes = '#ff2e63'
ativos = '#fe9801'

In [ ]:
# Agrupando casos recuperados, mortes e ativos por data
temp = casos_totais.groupby('state')['totalCases', 'recovered', 'deaths', 'actives'].sum().reset_index()

In [ ]:
# Remodela o df com variável e valor para ter a quantidade de recuperados, mortes e ativos.
temp = temp.melt(id_vars='state', value_vars=['totalCases', 'recovered', 'deaths', 'actives'],
                  var_name='Case', value_name='Count')

In [ ]:
temp.head()

In [ ]:
fig = px.area(temp,
              x='state',
              y='Count',
              color='Case',
              height=600,
              title='Casos ao longo do tempo por estados do Brasil',
              color_discrete_sequence = [total, recuperados, mortes, ativos])
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

Agrupando dados por data e cidade

In [ ]:
cidades.info()

In [ ]:
agrupado = cidades.groupby(['date', 'state', 'city'])['totalCases', 'newCases', 'deaths', 'newDeaths' ].sum().reset_index()

In [ ]:
agrupado.tail()

In [ ]:
agrupado.sort_values(by='totalCases', ascending=False)

In [ ]:
fig = px.choropleth(agrupado, 
                    locations='city',
                    locationmode='ISO-3',
                    color= np.log(agrupado['totalCases']),
                    hover_name='city', 
                    hover_data=['totalCases', 'deaths'],
                    animation_frame=agrupado['date'].dt.strftime('%d-%m-%y'),
                    title='Casos ao longo do tempo',
                    color_continuous_scale=px.colors.sequential.Magenta)
fig.update_layout(autosize=False, width=1200, height=800)
fig.show()

In [ ]:
!pip install plotly==4.5.2

In [ ]:
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.express as px

In [ ]:
completo = cidades[cidades['date']==max(cidades['date'])]

In [ ]:
fig = px.line(agrupado,
              x='date',
              y='deaths',
              color='state',
              height=600,
              title='Mortes confimadas',
              color_discrete_sequence = px.colors.qualitative.Dark2)
fig.show() 